# User CF + Item CF + SVD

In [3]:
import pandas as pd
import os

### Import data
The file "u.data" which contains the full dataset.
- 100,000 rating(1-5) from 943 users on 1682 movies
- Each user has rated at least 20 movies

In [4]:
data_dir = os.path.join('data', 'ml-100k')

In [5]:
data_dir

'data/ml-100k'

In [11]:
file_name = os.path.join(data_dir, 'u.data')

df = pd.read_csv(file_name, sep='\t', header = -1)

### full u dataset
- user id || item id || rating || timestape

Give the columns name to the dataset

In [26]:
df.columns = ['user_id', 'item_id', 'rating', 'itemstape']

In [151]:
df.head()

,user_id,item_id,rating,itemstape
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


### Split dataset(train 25%, test 75%) 

In [28]:
from sklearn.model_selection import train_test_split

train_data, test_data = train_test_split(df, test_size = 0.25)

### Rebuild the matrix(n x m) which n is number of users and m is the number of movies

In [146]:
n_users = df.user_id.unique().shape[0]

n_items = df.item_id.unique().shape[0]

print('Number of users = ' + str(n_users) + '| number of movies = '+ str(n_items))

Number of users = 943| number of movies = 1682


### Create two user-item matrix, one for training and another for testing

### Training dataset

In [147]:
import numpy as np

train_data_matrix = np.zeros((n_users, n_items))

train_data_matrix.shape

### Testing dataset

In [42]:
test_data_matrix = np.zeros((n_users, n_items))

In [152]:
train_data.head()

,user_id,item_id,rating,itemstape
88544,49,477,2,888067727
64210,193,286,4,889122906
77672,796,106,2,893194895
43188,416,431,4,886316164
52868,292,96,4,881103568


### insert the data into the train data matrix and test data matrix

Becuase user_id and item_id are start from 1, so the value in the matrix should be decread by 1

In [40]:
for line in train_data.itertuples():
    train_data_matrix[line[1] - 1, line[2] - 1] = line[3]

for line in test_data.itertuples():
    test_data_matrix[line[1] - 1, line[2] - 1] = line[3]

## Collaborative Flitering(Memory based)

There are two main parts of CF: User CF and Item CF. 

User CF: choose a specific user and based on the similarity of two users to recommend items(like: the person like these items also like that one)

Item CF: Find the persons who like the same item and these persons other like items. Calculate the similarity base on that


![title](data/User_base_and_item_base.jpeg)

### compute user and item similarity


In [51]:
import sklearn

##### The normal distance matrix in recommendation system we use the cosine similarity

User similarity:

$ S_{u}^{cosine}(U_{k},U_{a}) = \frac{U_{k}*U_{a}}{|U_{k}| |U_{a}|} = \frac{\sum{X_{k,m} * X_{a,m}}}{\sqrt{\sum{X_{k}^{2}}\sum{X^{2}_{a,m}}}} $

$ S_{u}^{cosine}(I_{m},I_{b}) = \frac{I_{m}*I_{b}}{|I_{m}| |I_{b}|} = \frac{\sum{X_{a,m} * X_{a,b}}}{\sqrt{\sum{X_{a,m}^{2}}\sum{X^{2}_{a,b}}}} $

In [59]:
user_similarity = sklearn.metrics.pairwise_distances(train_data_matrix, metric = "cosine")

item_similarity = sklearn.metrics.pairwise_distances(train_data_matrix.T, metric = "cosine")

#### The shape of the user similarity matrix becomes 943 x 943 

In [158]:
user_similarity.shape

(943, 943)

#### The shape of the item similarity matrix becomes 1682 x 1682

In [160]:
item_similarity.shape

(1682, 1682)

### Prediction based on either user or items

Prediction function is based on:
    $x_{k,m} = \overline{x_{k}} + \frac{\sum_{u_{a}}{sim_{u}(u_{k}, u_{a})(x_{a,m} - \overline{x_{u_{a}}})}}{\sum_{u_{a}}|sim_{u}(u_{k}, u_{a})|}$

![title](data/user_base.png)

##### The intuition here is that not everyone rating standard is same. 
##### For example, if user A like a movie he might give the movie 5 stars and others 3 or 4. But some picky people rarely give 5 star to the movie. 
##### So the algorithm use the similarity as the weight to predict rating

In [75]:
def predict(ratings, similarity, type = 'user'):
    if type == 'user':
        mean_user_rating = ratings.mean(axis = 1)
        ratings_diff = (ratings - mean_user_rating[:, np.newaxis])
        pred = mean_user_rating[:, np.newaxis] + similarity.dot(ratings_diff) / np.array(np.abs(similarity).sum(axis = 1)).T
    elif type == 'item':
        pred = ratings.dot(similarity) / np.array(np.abs(similarity).sum(axis = 1))
    return pred

In [79]:
user_prediction = predict(train_data_matrix, user_similarity, type='user')
item_prediction = predict(train_data_matrix, item_similarity, type='item')
# user_prediction = predict(train_data_matrix, item_similarity, type='user')

### Evaluation

##### The most popular way to evaluate the model is using RMSE(root-mean-square error)

$RMSE = \sqrt{\frac{1}{N}\sum(x_{i} - \hat{x_{i}})^{2}}$

In [80]:
from sklearn.metrics import mean_squared_error
from math import sqrt

In [81]:
def rmse(prediction, ground_truth):
    prediction = prediction[ground_truth.nonzero()].flatten()
    ground_truth = ground_truth[ground_truth.nonzero()].flatten()
    return sqrt(mean_squared_error(prediction, ground_truth))

In [82]:
print ('User-based CF RMSE: ' + str(rmse(user_prediction, test_data_matrix)))
print ('Item-based CF RMSE: ' + str(rmse(item_prediction, test_data_matrix)))

User-based CF RMSE: 3.128727161284137
Item-based CF RMSE: 3.4572013449040506


#### Conclusion:
Memory-based CF is easy to implement but the real problem is cold start, which means when the new users or data come

### Collabrative flitering base on SVD

In [101]:
sparsity = round(1.0 - len(df)/float(n_users*n_items),3)

In [104]:
print('The sparsity level of the MovieLens 100K is ' + str(sparsity * 100) + '%')

The sparsity level of the MovieLens 100K is 93.7%


#### Use scipy svds

In [105]:
import scipy.sparse as sp
from scipy.sparse.linalg import svds

In [131]:
u, s, vt = svds(train_data_matrix, k = 10)

In [132]:
s_diag_matrix = np.diag(s)

In [133]:
X_pred = np.dot(np.dot(u, s_diag_matrix), vt)

In [143]:
print('User-based CF RMSE: ' + str(rmse(X_pred, test_data_matrix)))

User-based CF RMSE: 2.6833982353839883


#### Use sklearn

In [135]:
from sklearn.decomposition import TruncatedSVD

In [136]:
svd = TruncatedSVD(n_components=5, n_iter=7, random_state=42)

In [137]:
X_sk_pred = svd.fit(train_data_matrix)